In [1]:
# workout_snp.py

# Dynamically manage open orders
from z_helper import *
util.startLoop()

ib = get_connected('snp', 'live')


In [4]:
trds = ib.trades()

In [5]:
trds[0]

Trade(contract=Option(conId=347073549, symbol='HD', lastTradeDateOrContractMonth='20190816', strike=170.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='HD    190816P00170000', tradingClass='HD', comboLegs=[]), order=Order(permId=31249884, action='SELL', totalQuantity=1, orderType='LMT', lmtPrice=1.35, auxPrice=0.0, tif='DAY', ocaType=3, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U8898867', clearingIntent='IB', orderComboLegs=[], conditions=[], softDollarTier=SoftDollarTier(), cashQty=0.0, dontUseAutoPriceForHedge=True, filledQuantity=0.0, refFuturesConId=2147483647, shareholder='Not an insider or substantial shareholder', parentPermId=9223372036854775807), orderStatus=OrderStatus(status='Cancelled'), fills=[], log=[])

In [23]:
df_trades = util.df(t.contract for t in trds).join(util.df(t.order for t in trds)).join(util.df(t.orderStatus for t in trds), lsuffix='_')
cols = ['localSymbol', 'secType', 'conId', 'symbol', 'lastTradeDateOrContractMonth', 'strike', 'right', 
        'multiplier', 'orderId', 'permId', 'action', 'totalQuantity', 'orderType', 'lmtPrice', 
        'status', 'filled', 'remaining', 'avgFillPrice', 'lastFillPrice']

In [24]:
df_trades[cols]

,localSymbol,secType,conId,symbol,lastTradeDateOrContractMonth,strike,right,multiplier,orderId,permId,action,totalQuantity,orderType,lmtPrice,status,filled,remaining,avgFillPrice,lastFillPrice
0,HD 190816P00170000,OPT,347073549,HD,20190816,170.0,P,100,0,0,SELL,1,LMT,1.35,Cancelled,0,0,0.0,0.0
1,AMGN 190802P00155000,OPT,370932454,AMGN,20190802,155.0,P,100,0,0,SELL,1,LMT,0.90,Cancelled,0,0,0.0,0.0
2,FB 190802P00145000,OPT,370979681,FB,20190802,145.0,P,100,0,0,SELL,1,LMT,0.63,Cancelled,0,0,0.0,0.0
3,LOW 190816C00130000,OPT,372303509,LOW,20190816,130.0,C,100,0,0,SELL,1,LMT,0.24,Cancelled,0,0,0.0,0.0
4,ADBE 190726P00245000,OPT,370056355,ADBE,20190726,245.0,P,100,0,0,SELL,1,LMT,1.01,Cancelled,0,0,0.0,0.0
5,CL 190712C00077000,OPT,368821701,CL,20190712,77.0,C,100,0,0,SELL,1,LMT,0.10,Cancelled,0,0,0.0,0.0
6,BRKB 190809C00240000,OPT,372885150,BRK B,20190809,240.0,C,100,0,0,SELL,1,LMT,2.32,Cancelled,0,0,0.0,0.0
7,BRKB 190802C00240000,OPT,370931017,BRK B,20190802,240.0,C,100,0,0,SELL,1,LMT,1.81,Cancelled,0,0,0.0,0.0
8,FB 190726C00232500,OPT,372912147,FB,20190726,232.5,C,100,0,0,SELL,1,LMT,0.61,Cancelled,0,0,0.0,0.0
9,AMGN 190816P00150000,OPT,354959365,AMGN,20190816,150.0,P,100,0,0,SELL,1,LMT,1.20,Cancelled,0,0,0.0,0.0
